In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from io import BytesIO
from selenium.webdriver.chrome.options import Options
import requests
import fitz

def Korea_Bank_News_Text(page=5):
    infos = []
    driver, links = Korea_Bank_News_Links(page=page)

    for link in links:
        info = {}
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        item = soup.select_one(".down > dd > ul > li > a")

        pdf_link = "https://www.bok.or.kr" + item["href"]
        pdf_title = item["title"][:item["title"].rfind(".")]
        response = requests.get(pdf_link)
        # 예외처리
        response.raise_for_status()

        doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")
        text = "\n".join(page.get_text() for page in doc)
        info[pdf_title] = text
        infos.append(info)
    
    driver.quit()
    
    return infos

def Korea_Bank_News_Links(page=5):
    # ChromeDriver 경로 명시
    chromedriver_path = "/usr/local/bin/chromedriver"  # ChromeDriver 경로
    chrome_path = "/usr/bin/google-chrome"  # Chrome 브라우저 경로 (시스템에 따라 다를 수 있음)

    # Chrome 옵션 설정
    options = Options()
    options.add_argument("--headless")  # GUI 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = chrome_path  # Chrome 브라우저 경로 설정

    # ChromeDriver 서비스 설정
    service = Service(chromedriver_path)

    # WebDriver 생성
    driver = webdriver.Chrome(service=service, options=options)

    links = []

    for i in range(1, page+1):

        url = f"https://www.bok.or.kr/portal/singl/newsData/list.do?pageIndex={i}&targetDepth=2&menuNo=200080&syncMenuChekKey=1&depthSubMain=&subMainAt=&searchCnd=1&searchKwd=&depth2=200080&date=&sdate=&edate=&sort=1&pageUnit=10"
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        items = soup.select(".set > a")

        for item in items:
            if item["href"] == "#":
                continue

            links.append("https://www.bok.or.kr" + item["href"])

    return driver, links

In [46]:
Korea_Bank_News_Text()

[{'(현지정보) 美 2025.6월 고용지표 내용 및 뉴욕 금융시장 반응_f': '1/3\n2025. 7. 3(목)\n현지정보\n한국은행뉴욕사무소\n美 2025.6월 고용지표 내용 및 뉴욕 금융시장 반응\n1. 고용지표 내용\n□6월중 비농업부문 취업자수(Non-farm Payrolls)는 전월대비14.7만명 증가\n[Bloomberg 시장예상치 +10.6만명]\no 민간부문취업자수는7.4만명 증가[예상 +10.0만명]하였으며 정부부문 취\n업자수는 7.3만명 증가\n－세부 업종별로는 교육‧의료(+5.1만명), 여가‧음식숙박(+2.0만명) 등은 증가하였\n으나 전문기업서비스(-0.7만명), 제조업(-0.7만명) 등은 감소\no 직전 2개월 취업자수는 1.6만명 상향조정(5월 +0.5만명, 4월 +1.1만명)\n□경제활동참가율(62.4% →62.3%)[예상 62.4%]은 전월에 비해 하락하였으며, 실업률\n(4.2% →4.1%)[예상 4.3%]도 전월에 비해 하락\no 가계조사에서 경제활동인구가 13.0만명 감소한 가운데, 이 중 취업자수*는 \n9.3만명 증가하였으며 실업자수는 22.2만명 감소\n* 기업조사 기준 조정치(농림업 및 자영업 제외, 다중취업 추가)는 전월대비 75.1만명 증가\no U-6 실업률*은 7.7%로 전월(7.8%)에 비해 하락\n* 최근 4주간만 구직활동을 하지 않거나, 경제적 이유의 비자발적 시간제 근로자를 실업자로 간주\n□시간당 평균임금상승률은 전월대비(0.4% →0.2%)[예상0.3%] 및전년동월대비\n(3.8% →3.7%)[예상3.8%] 모두 축소\no 주당 평균노동시간(34.3시간→34.2시간)[예상34.3시간]은전월대비 감소\n주요 부문별 非농업 취업자수 증감\n실업률 및 시간당 평균임금 상승률\n(만명)\n3월\n4월\n5월\n3개월 평균\n非농업 전체\n15.8\n14.4\n14.7\n15.0\n-민간부문\n13.3\n13.7\n7.4\n11.5\n(건  설)\n0.0\n0.6\n1.5\n0.7\n(제  조)\n0.0\